# MLClass. "Прикладной анализ данных"
# Модуль "Машинное обучение с помощью Python"
<img src="../img/mlclass_logo.jpg" height="240" width="240">
## Автор материала: преподаватель ФКН НИУ ВШЭ Кашницкий Юрий
Материал распространяется на условиях лицензии <a href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-Share Alike 4.0</a>. Можно использовать в любых целях, но с обязательным упоминанием автора курса и аффилиации.

# Урок 6. Нейронные сети. Бустинг. Смешивание алгоритмов. Стекинг.
## Часть 1. Решение Otto Group Product Classification Challenge с помощью  nolearn/lasagne

Переведено из <a href="https://github.com/ottogroup">этого</a> репозитория. 

**Подключение библиотек**

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.nonlinearities import softmax
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

**Вспомогательные функции**

In [3]:
def load_train_data(path):
    df = pd.read_csv(path)
    X = df.values.copy()
    np.random.shuffle(X)
    X, labels = X[:, 1:-1].astype(np.float32), X[:, -1]
    encoder = LabelEncoder()
    y = encoder.fit_transform(labels).astype(np.int32)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y, encoder, scaler

In [4]:
def load_test_data(path, scaler):
    df = pd.read_csv(path)
    X = df.values.copy()
    X, ids = X[:, 1:].astype(np.float32), X[:, 0].astype(str)
    X = scaler.transform(X)
    return X, ids

In [5]:
def make_submission(clf, X_test, ids, encoder, 
                    name='my_neural_net_submission.csv'):
    y_prob = clf.predict_proba(X_test)
    with open(name, 'w') as f:
        f.write('id,')
        f.write(','.join(encoder.classes_))
        f.write('\n')
        for id, probs in zip(ids, y_prob):
            probas = ','.join([id] + map(str, probs.tolist()))
            f.write(probas)
            f.write('\n')
    print("Wrote submission to file {}.".format(name))

**Загрузка данных**

In [6]:
X, y, encoder, scaler = load_train_data('../data/otto_train.csv')

In [7]:
X_test, ids = load_test_data('../data/otto_test.csv', scaler)

In [8]:
num_classes = len(encoder.classes_)
num_features = X.shape[1]

**Обучение нейронной сети**

In [9]:
layers0 = [('input', InputLayer),
           ('dense0', DenseLayer),
           ('dropout', DropoutLayer),
           ('dense1', DenseLayer),
           ('output', DenseLayer)]

In [10]:
net0 = NeuralNet(layers=layers0,
                 
                 input_shape=(None, num_features),
                 dense0_num_units=200,
                 dropout_p=0.5,
                 dense1_num_units=200,
                 output_num_units=num_classes,
                 output_nonlinearity=softmax,
                 
                 update=nesterov_momentum,
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 eval_size=0.2,
                 verbose=1,
                 max_epochs=20)

//anaconda/lib/python2.7/site-packages/nolearn/lasagne/base.py:206: UserWarning: The 'eval_size' argument has been deprecated, please use the 'train_split' parameter instead, e.g.
train_split=TrainSplit(eval_size=0.4)
  warn("The 'eval_size' argument has been deprecated, please use "


In [11]:
%%time
net0.fit(X, y)

# Neural Network with 60809 learnable parameters

## Layer information

  #  name       size
---  -------  ------
  0  input        93
  1  dense0      200
  2  dropout     200
  3  dense1      200
  4  output        9

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  -----
      1       0.92033       0.66250      1.38918      0.75145  2.58s
      2       0.70799       0.61887      1.14401      0.76622  2.91s
      3       0.66972       0.59583      1.12401      0.77033  2.99s
      4       0.64441       0.58727      1.09729      0.77578  3.26s
      5       0.62675       0.57321      1.09339      0.77960  3.61s
      6       0.61363       0.56452      1.08699      0.78162  3.56s
      7       0.60270       0.55902      1.07813      0.78561  3.53s
      8       0.59747       0.55123      1.08388      0.78834  3.36s
      9       0.58576       0.54748      1.06993      0.78846  3.09s
     10       0.57831  

NeuralNet(X_tensor_type=None,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x1092532d0>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x109253250>,
     custom_score=None, dense0_num_units=200, dense1_num_units=200,
     dropout_p=0.5, input_shape=(None, 93),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('dense0', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout', <class 'lasagne.layers.noise.DropoutLayer'>), ('dense1', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=20, more_params={},
     objective=<function objective at 0x109254140>,
     objective_loss_function=<function categorical_crossentropy at 0x1090ba758>,
     on_batch_finished=[],
     on_epoch_finished=[<nolearn.lasagne.handlers.PrintLog instance at 0x10927cb90>],
     on_training_finished=[],
     on_training_started=[<nolearn.lasagne.handlers.PrintLayerInfo instan

**Предсказание для тестовых данных**

In [12]:
make_submission(net0, X_test, ids, encoder, 
                name="../output/lasagne_otto.csv")

Wrote submission to file ../output/lasagne_otto.csv.


**У такой посылки на Kaggle результат multi_log_loss=0.52814.**

### Ссылки

- <a href="https://pythonhosted.org/nolearn/">Документация</a> Nolearn
- <a href="https://www.kaggle.com/c/otto-group-product-classification-challenge">Соревнование</a> Kaggle компании Otto по категоризации продуктов
- <a href="https://github.com/ottogroup">Репозиторий</a> компании Otto
- <a href="https://github.com/dnouri/nolearn">Репозиторий</a> Nolearn
- <a href="https://github.com/benanne/Lasagne">Репозиторий</a> Lasagne
- <a href="http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/">Тьюториал</a> Nolearn/lasagne по сверточным нейронным сетям. 